# TCI Grid for cities

Running the pipeline for specific cities

In [29]:
import os
import yaml
import itertools
from glob import glob

## Configuration file

In [30]:
config = yaml.load(open("config_cities.yaml"), Loader=yaml.FullLoader)
#config

In [31]:
countries = [k for k, v in config["cities_names"].items()]
cities_lists = [config["cities_names"][k] for k, v in config["cities_names"].items()]
cities_configured = [item for sublist in cities_lists for item in sublist]

## Evaluation of pipeline

In [32]:
%%bash
ls ~/shared/spd-sdv-omitnik-waze/corona/allhist/tci_grid/ | wc -l

51


In [33]:
path_write = '/home/soniame/shared/spd-sdv-omitnik-waze/corona/allhist/tci_grid'
cities_done = os.listdir(path_write)
len(cities_done)

51

### Warning!

Cities without a result in S3 shared folder

In [38]:
cities_missing = list()
for city in cities_configured:
    if city not in cities_done:
        print(city)
        cities_missing.append(city)

natal
florianopolis
montevideo


## Testing the pipeline per country

In [35]:
countries

['Argentina', 'Brasil', 'Chile', 'Colombia', 'Mexico', 'Peru', 'Uruguay']

In [44]:
def fun_cities_txt(cities_list):
    cities_txt = '" "'.join(cities_list)

    return(cities_txt)

def fun_sh_text(cities_txt):
    
    sh_script = """
# Running pipeline per city

declare -a cities=("%s")

for city in ${cities[*]};
do 
    echo $city
    python src/entrypoint.py single --config_path='configs/config-grid.yaml' --dependency_graph_path='configs/dependency-graph-grid.yaml' --slug='allhist' --region_def=$city
done
echo 'All done'
    """ % (cities_txt)
    
    return(sh_script)

In [47]:
def fun_sh_script(country):

    sh_text = fun_sh_text(fun_cities_txt(cities_list = config['cities_names'][country]))
    
    file_name = f"grid_{country.lower()}.sh"
    file1 = open(file_name, "w")
    file1.write(sh_text) 
    file1.close() 
    
    return None

### Create bash file per country

In [48]:
fun_sh_script(country = 'Mexico')

### Create bash file for specific cities

In [49]:
cities_missing

['natal', 'florianopolis', 'montevideo']

In [51]:
sh_text = fun_sh_text(fun_cities_txt(cities_missing))

file_name = "grid_missing.sh"
file1 = open(file_name, "w")
file1.write(sh_text) 
file1.close() 